# 📓 Notebook 2 – Analysis of single animal (top-view mouse)
## 1. Introduction & objectives

In this notebook, we will analyze pose estimation outputs generated with SuperAnimal ModelZoo on 5-minute top-view mouse video.

**Learning goals:**


--- 
**Instructions**

This notebook mixes pre-filled code cells (nothing to change) and coding exercises that you will complete.

👉 Here’s how to work through it:
1. Read carefully each section before running the cells.
2.	When a cell requires you to code, you’ll see a TODO comment.
3.	The TODO will tell you how many lines of code you are expected to write.
4.	Write your code only between the markers:
    
```python
# >>>>>>>>>>>>>>>>>>>
# your code goes here
# <<<<<<<<<<<<<<<<<<<
```

✋ Do not edit anything outside these markers.

⚡ After finishing the course, feel free to experiment and modify the notebook as you like!

✨ Example

What you will see in the notebook:

```python
# >>>>>>>>>>>>>>>>>>>
# TODO (2 lines): compute the duration of the video and print it 
# variables: frame_count, fps
# YOUR CODE: duration = 
# YOUR CODE: print(...)
# <<<<<<<<<<<<<<<<<<< 
```

What you are expected to write: 

```python
# >>>>>>>>>>>>>>>>>>>
# TODO (2 lines): compute the duration of the video and print it 
# variables: frame_count, fps
duration = frame_count / fps
print(f"Duration (s):", duration)
# <<<<<<<<<<<<<<<<<<< 
```
---
  

<img src="https://raw.githubusercontent.com/LizbethMG-Teaching/pose2behav-book/main/assets/notebook2-cover.png" width="50%">

**Narrative**

In this notebook, you will analyze pose-estimation data from a freely-moving mouse. 

You will follow **Pixel** 🐭💛, a little mouse exploring an arena.
Using pose data captured by SuperAnimal, we’ll uncover how Pixel moves, rests, and explores. 

The goal is to transform raw x,y coordinates into quantitative descriptions of movement and space use, similar to what behavioral neuroscientists do when studying exploration and locomotion.

For this exercise you will have access to the data in form of a .h5 file that has been already pre-processed as in the previous notebook. 

--- 

## 2. Data Loading & Format Inspection

### 2.1 Download data (prefilled)

**📋 Instructions:**
1. Run the code cell below to download a dataset file from Google Drive (with gdown), save it locally (path depends on Colab vs local), and verify the download.